### Attention: s'assurer que le fichier DataSets.py présent dans le chemin indiqué est bien celui du fichier nn3! Sinon risque de crash du code;

In [1]:
import tensorflow as tf
import numpy as np
print("TensorFlow version: ",format(tf.__version__))
from pathlib import Path
import DataSets as ds

TensorFlow version:  2.0.0


### chargement des données

In [2]:
LoadModel = False
experiment_size = 10
train = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_%dk.bin"%experiment_size), \
                   Path("/home/romain/jupyter_mla/Databases/gender_%dk.bin"%experiment_size), 1000*experiment_size)
test = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_test10k.bin"), \
     Path("/home/romain/jupyter_mla/Databases/gender_test10k.bin"), 10000)

nb data =  10000
nb data =  10000


### classes pour définir les couches et la structure du réseau

In [3]:
class fc_layer(tf.Module):
    def __init__(self, name, input_dim, output_dim):
        self.scope_name = name
        w_init = tf.random.truncated_normal([input_dim, output_dim], stddev=0.1)
        self.w = tf.Variable(w_init)
        print('w      ', self.w.shape)
        b_init = tf.constant(0.0, shape=[output_dim])
        self.b = tf.Variable(b_init)
        print('b      ', self.b.get_shape())


    def __call__(self, x, log_summary):
        if log_summary:
            with tf.name_scope(self.scope_name) as scope:
                tf.summary.scalar("mean w", tf.reduce_mean(self.w))
                tf.summary.scalar("max w", tf.reduce_max(self.w))
                tf.summary.histogram("w", self.w)
                tf.summary.scalar("mean b", tf.reduce_mean(self.b))
                tf.summary.scalar("max b", tf.reduce_max(self.b))
                tf.summary.histogram("b", self.b)
        return tf.matmul(x, self.w) + self.b

In [4]:
class SimpleNet(tf.Module):
    def __init__(self, input_dim):
        self.fc1 = fc_layer('fc1', input_dim, 50)
        self.fc2 = fc_layer('fc2', 50, 2)

    def __call__(self, x, log_summary):
        x = self.fc1(x, log_summary)
        x = tf.nn.sigmoid(x)
        x = self.fc2(x, log_summary)
        return x

In [5]:
def train_one_iter(model, optimizer, image, label, log_summary):
    with tf.GradientTape() as tape:
        y = model(image,log_summary)
        loss = tf.reduce_sum(tf.square(y - label))
        if log_summary:
            tf.summary.scalar('loss', loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

### Résumé du modèle

In [6]:
print ("-----------------------------------------------------")
print ("----------------------- %dk -------------------------"%experiment_size)
print ("-----------------------------------------------------")

-----------------------------------------------------
----------------------- 10k -------------------------
-----------------------------------------------------


In [7]:
train_summary_writer = tf.summary.create_file_writer('logs')
optimizer = tf.optimizers.SGD(1e-4)
simple_v2 = SimpleNet(train.dim)

w       (2304, 50)
b       (50,)
w       (50, 2)
b       (2,)


In [8]:
for iter in range(5000):
    tf.summary.experimental.set_step(iter)
    ima, lab = train.NextTrainingBatch()
    with train_summary_writer.as_default():
        loss = train_one_iter(simple_v2, optimizer, ima, lab, iter % 10 == 0)
    if iter % 100 == 0:
        print("iter= %6d - loss= %f" % (iter, loss))
    if iter % 500 == 0:
        with train_summary_writer.as_default():
            acc1 = train.mean_accuracy(simple_v2)*100
            acc2 = test.mean_accuracy(simple_v2)*100
            print("iter= %6d accuracy - train= %.2f%% - test= %.2f%%" % (iter, acc1,acc2))

iter=      0 - loss= 364.795807
iter=      0 accuracy - train= 48.01% - test= 54.08%
iter=    100 - loss= 43.697674
iter=    200 - loss= 3.933403
iter=    300 - loss= 5.380292
iter=    400 - loss= 70.471382
iter=    500 - loss= 42.113567
iter=    500 accuracy - train= 60.78% - test= 64.67%
iter=    600 - loss= 77.402618
iter=    700 - loss= 39.394745
iter=    800 - loss= 34.234615
iter=    900 - loss= 6.834790
iter=   1000 - loss= 20.229486
iter=   1000 accuracy - train= 58.92% - test= 55.98%
iter=   1100 - loss= 54.763031
iter=   1200 - loss= 52.374474
iter=   1300 - loss= 12.051232
iter=   1400 - loss= 183.049240
iter=   1500 - loss= 39.735825
iter=   1500 accuracy - train= 60.22% - test= 57.41%
iter=   1600 - loss= 10.685817
iter=   1700 - loss= 51.838348
iter=   1800 - loss= 34.396255
iter=   1900 - loss= 68.069977
iter=   2000 - loss= 31.615200
iter=   2000 accuracy - train= 62.67% - test= 66.30%
iter=   2100 - loss= 5.991741
iter=   2200 - loss= 23.060081
iter=   2300 - loss= 18.